In [23]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [2]:
xgb.__version__

'2.1.1'

In [3]:
df = pd.read_csv("data/train.csv")
subdf = pd.read_csv("data/test.csv")

In [4]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [5]:
df.drop("Id", axis=1, inplace=True)
subdf.drop("Id", axis=1, inplace=True)

In [6]:
X = df.drop('SalePrice', axis =1).copy()
y = df['SalePrice'].copy()

In [55]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     float64
 6   LotShape       1460 non-null   int64  
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   int64  
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [19]:

col = X.columns
for c in col:
    if X[c].dtype==object and len(X[c].unique())!=len(subdf[c].unique()):
        #print(f"{c} has different number of classes in train and test")
        setx = set(X[c].unique())
        setsubdf = set(subdf[c].unique())
        print(f"{c} values difference between train and test are {setx-setsubdf}")
        

MSZoning values difference between train and test are set()
Condition2 values difference between train and test are {'RRAn', 'RRNn', 'RRAe'}
HouseStyle values difference between train and test are {'2.5Fin'}
RoofMatl values difference between train and test are {'Membran', 'Metal', 'ClyTile', 'Roll'}
Exterior1st values difference between train and test are {'ImStucc', 'Stone'}
Heating values difference between train and test are {'Floor', 'OthW'}
Electrical values difference between train and test are {'Mix', nan}
MiscFeature values difference between train and test are {'TenC'}
SaleType values difference between train and test are set()


In [12]:
subdf

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [7]:
# there are missing values but no nan as being shown in pandas
nullcolx= []
nullcolsub = []
for c in X.columns:
    if X[c].isnull().values.any():
        nullcolx.append(c)
    if subdf[c].isnull().values.any():
        nullcolsub.append(c)



In [8]:
for i in nullcolx:
    if i not in nullcolsub:
        print(i)

Electrical


In [9]:
for i in nullcolsub:
    if i not in nullcolx:
        print(i)

MSZoning
Utilities
Exterior1st
Exterior2nd
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
BsmtFullBath
BsmtHalfBath
KitchenQual
Functional
GarageCars
GarageArea
SaleType


In [10]:
for i in nullcolsub:
    if i in nullcolx:
        print(i)

LotFrontage
Alley
MasVnrType
MasVnrArea
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageQual
GarageCond
PoolQC
Fence
MiscFeature


In [11]:
# feature to be encoded
int_encode_mapping = {
    "Alley":{"Grvl":2, "Pave":1 ,"NA":0},
    "LotShape":{"Reg":4, "IR1":3, "IR2":2, "IR3":1},
    "Utilities":{"AllPub":4, "NoSewr":3, "NoSeWa":2, "ELO":1},
    "ExterQual":{"Ex":5,"Gd":4,"TA":3,"Fa":2, "Po":1},
    "ExterCond":{"Ex":5,"Gd":4,"TA":3,"Fa":2, "Po":1},
    "BsmtQual":{"Ex":5,"Gd":4,"TA":3,"Fa":2, "Po":1},
    "BsmtCond":{"Ex":5,"Gd":4,"TA":3,"Fa":2, "Po":1},
    "BsmtExposure":{"Gd":4,"Av":3,"Mn":2, "No":1},
    "BsmtFinType1":{"GLQ":6,"ALQ":5,"BLQ":4, "Rec":3, "LwQ":2, "Unf":1},
    "BsmtFinType2":{"GLQ":6,"ALQ":5,"BLQ":4, "Rec":3, "LwQ":2, "Unf":1},
    "HeatingQC":{"Ex":5,"Gd":4,"TA":3,"Fa":2, "Po":1},
    "KitchenQual":{"Ex":5,"Gd":4,"TA":3,"Fa":2, "Po":1},
    "Functional":{"Typ":8, "Min1":7, "Min2":6, "Mod":5, "Maj1":4, "Maj2":3, "Sev":2, "Sal":1},
    "FireplaceQu":{"Ex":5,"Gd":4,"TA":3,"Fa":2, "Po":1},
    "GarageQual":{"Ex":5,"Gd":4,"TA":3,"Fa":2, "Po":1},
    "GarageCond":{"Ex":5,"Gd":4,"TA":3,"Fa":2, "Po":1},
    "PoolQC":{"Ex":5,"Gd":4,"TA":3,"Fa":2, "Po":1}
}

In [12]:
X.update(X[list(int_encode_mapping)].apply(lambda col: col.map(int_encode_mapping[col.name])))
subdf.update(subdf[list(int_encode_mapping)].apply(lambda col: col.map(int_encode_mapping[col.name])))


In [13]:
intcol = int_encode_mapping.keys()
for c in intcol:
    print(c)
    X[c]=pd.to_numeric(X[c])
    subdf[c] = pd.to_numeric(subdf[c])

Alley
LotShape
Utilities
ExterQual
ExterCond
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
HeatingQC
KitchenQual
Functional
FireplaceQu
GarageQual
GarageCond
PoolQC


In [14]:
col = X.columns
for c in col:
    print(c, X[c].dtype)

MSSubClass int64
MSZoning object
LotFrontage float64
LotArea int64
Street object
Alley float64
LotShape int64
LandContour object
Utilities int64
LotConfig object
LandSlope object
Neighborhood object
Condition1 object
Condition2 object
BldgType object
HouseStyle object
OverallQual int64
OverallCond int64
YearBuilt int64
YearRemodAdd int64
RoofStyle object
RoofMatl object
Exterior1st object
Exterior2nd object
MasVnrType object
MasVnrArea float64
ExterQual int64
ExterCond int64
Foundation object
BsmtQual float64
BsmtCond float64
BsmtExposure float64
BsmtFinType1 float64
BsmtFinSF1 int64
BsmtFinType2 float64
BsmtFinSF2 int64
BsmtUnfSF int64
TotalBsmtSF int64
Heating object
HeatingQC int64
CentralAir object
Electrical object
1stFlrSF int64
2ndFlrSF int64
LowQualFinSF int64
GrLivArea int64
BsmtFullBath int64
BsmtHalfBath int64
FullBath int64
HalfBath int64
BedroomAbvGr int64
KitchenAbvGr int64
KitchenQual int64
TotRmsAbvGrd int64
Functional int64
Fireplaces int64
FireplaceQu float64
GarageTy

In [24]:
objectcol = [c for c in X.columns if X[c].dtype==object]
#objectcol

In [42]:
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore', sparse_output=False),
['MSZoning','Street','LandContour','LotConfig','LandSlope','Neighborhood','Condition1','Condition2',
 'BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Foundation',
 'Heating','CentralAir','Electrical','GarageType','GarageFinish','PavedDrive','Fence','MiscFeature',
 'SaleType','SaleCondition']))

In [43]:
# Transforming
transformed = transformer.fit_transform(X)
# Transformating back
transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())

transformed_df.index = X.index
X_train = pd.concat([X, transformed_df], axis=1)
# Dropping old categorical columns
X_train.drop(objectcol, axis=1, inplace=True)
X_train.head()


,MSSubClass,LotFrontage,LotArea,Alley,LotShape,Utilities,OverallQual,OverallCond,YearBuilt,YearRemodAdd,...,onehotencoder__SaleType_ConLw,onehotencoder__SaleType_New,onehotencoder__SaleType_Oth,onehotencoder__SaleType_WD,onehotencoder__SaleCondition_Abnorml,onehotencoder__SaleCondition_AdjLand,onehotencoder__SaleCondition_Alloca,onehotencoder__SaleCondition_Family,onehotencoder__SaleCondition_Normal,onehotencoder__SaleCondition_Partial
0,60,65.0,8450,NaN,4,4,7,5,2003,2003,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,20,80.0,9600,NaN,4,4,6,8,1976,1976,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,60,68.0,11250,NaN,3,4,7,5,2001,2002,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,70,60.0,9550,NaN,3,4,7,5,1915,1970,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,60,84.0,14260,NaN,3,4,8,5,2000,2000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [44]:
# Transforming
transformed = transformer.transform(subdf)
# Transformating back
transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
print(transformed_df.index)
transformed_df.index = subdf.index
subdf_trans = pd.concat([subdf, transformed_df], axis=1)
# Dropping old categorical columns
subdf_trans.drop(objectcol, axis=1, inplace=True)
subdf_trans.head()

RangeIndex(start=0, stop=1459, step=1)


,MSSubClass,LotFrontage,LotArea,Alley,LotShape,Utilities,OverallQual,OverallCond,YearBuilt,YearRemodAdd,...,onehotencoder__SaleType_ConLw,onehotencoder__SaleType_New,onehotencoder__SaleType_Oth,onehotencoder__SaleType_WD,onehotencoder__SaleCondition_Abnorml,onehotencoder__SaleCondition_AdjLand,onehotencoder__SaleCondition_Alloca,onehotencoder__SaleCondition_Family,onehotencoder__SaleCondition_Normal,onehotencoder__SaleCondition_Partial
0,20,80.0,11622,NaN,4,4.0,5,6,1961,1961,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,20,81.0,14267,NaN,3,4.0,6,6,1958,1958,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,60,74.0,13830,NaN,3,4.0,5,5,1997,1998,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,60,78.0,9978,NaN,3,4.0,6,6,1998,1998,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,120,43.0,5005,NaN,3,4.0,8,5,1992,1992,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [45]:
X_train,X_test, y_train, y_test = train_test_split(X_train,y,random_state=42, shuffle=True)

In [57]:
reg_xgb  = xgb.XGBRegressor(n_estimators = 1000,
                            max_depth = 7,
                            eta=0.1,
                            objective="reg:squaredlogerror",
                             early_stopping_rounds=10,
                             subsample=0.7, colsample_bytree=0.8,
                             seed=42)
reg_xgb.fit(X_train,
           y_train,
           verbose=True,
           
           
           eval_set=[(X_test,y_test)])

[0]	validation_0-rmsle:11.26883
[1]	validation_0-rmsle:11.18104
[2]	validation_0-rmsle:11.09331
[3]	validation_0-rmsle:11.00566
[4]	validation_0-rmsle:10.91807
[5]	validation_0-rmsle:10.83055
[6]	validation_0-rmsle:10.74310
[7]	validation_0-rmsle:10.65574
[8]	validation_0-rmsle:10.56845
[9]	validation_0-rmsle:10.48124
[10]	validation_0-rmsle:10.39414
[11]	validation_0-rmsle:10.30713
[12]	validation_0-rmsle:10.22024
[13]	validation_0-rmsle:10.13346
[14]	validation_0-rmsle:10.04682
[15]	validation_0-rmsle:9.96034
[16]	validation_0-rmsle:9.87403
[17]	validation_0-rmsle:9.78790
[18]	validation_0-rmsle:9.70199
[19]	validation_0-rmsle:9.61629
[20]	validation_0-rmsle:9.53086
[21]	validation_0-rmsle:9.44568
[22]	validation_0-rmsle:9.36086
[23]	validation_0-rmsle:9.27654
[24]	validation_0-rmsle:9.19256
[25]	validation_0-rmsle:9.10925
[26]	validation_0-rmsle:9.02639
[27]	validation_0-rmsle:8.94428
[28]	validation_0-rmsle:8.86289
[29]	validation_0-rmsle:8.78250
[30]	validation_0-rmsle:8.70297
[31

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=10,
             enable_categorical=False, eta=0.1, eval_metric=None,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [50]:
sub = reg_xgb.predict(subdf_trans)

In [54]:
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [52]:
len(sub)

1459